In [ ]:
import os
from pathlib import Path

import numpy as np
import xarray as xr

In [ ]:
dirpath = os.path.join(Path.home(), "FjordsSim_data", "sognefjord")
ds = xr.open_dataset(os.path.join(dirpath, "sogn800LL_2018.nc"))

In [ ]:
ds["depth"] = -ds.depth

In [ ]:
ds = ds.rename({"Ix": "Nx", "Jy": "Ny", "depth": "Nz", "temp": "T", "salt": "S", "U": "u", "V": "v"})

In [ ]:
ds

In [ ]:
ds.h.plot()

In [ ]:
ds.T.isel(time=0).sel(Nz=-3).plot()

In [ ]:
ds.S.isel(time=0).sel(Nz=-3).plot()

In [ ]:
z_faces = [
      -1200.0,
      -800.0,
      -300.0,
      -80.0,
      -30.0,
      -10.0,
       -5.0,
        0.0
]

In [ ]:
z_centers = [(z_faces[i] + z_faces[i+1]) / 2 for i in range(len(z_faces) - 1)]

In [ ]:
z_centers

In [ ]:
ds = ds.interp(Nz=z_centers)

In [ ]:
ds

In [ ]:
ds.S.isel(time=0).isel(Nz=-1).plot()

In [ ]:
T_lambda_data = np.full(ds.T.shape, -999, dtype=np.float32)
T_lambda_data[:, :, :, :] = 1 / 60 / 60 / 12
T_lambda_da = xr.DataArray(
    T_lambda_data,
    dims=("time", "Nz", "Ny", "Nx"),
    coords={dim: ds.coords[dim] for dim in ["time", "Nz", "Ny", "Nx"]},  # Copy coords
    name="T_lambda"
)
ds["T_lambda"] = T_lambda_da

In [ ]:
S_lambda_data = np.full(ds.S.shape, -999, dtype=np.float32)
S_lambda_data[:, :, :, :] = 1 / 60 / 60 / 12
S_lambda_da = xr.DataArray(
    S_lambda_data,
    dims=("time", "Nz", "Ny", "Nx"),
    coords={dim: ds.coords[dim] for dim in ["time", "Nz", "Ny", "Nx"]},  # Copy coords
    name="S_lambda"
)
ds["S_lambda"] = S_lambda_da

In [ ]:
ds["T"] = ds["T"].astype(np.float32)
ds["S"] = ds["S"].astype(np.float32)

In [ ]:
ds

In [ ]:
mask = np.zeros_like(ds["T"], dtype=bool)
mask[:, :, [0, -1], :] = True
mask[:, :, :, [0, -1]] = True

In [ ]:
ds["T"] = xr.where(mask, ds["T"], np.nan)
ds["T_lambda"] = xr.where(mask, ds["T_lambda"], np.nan)
ds["S"] = xr.where(mask, ds["S"], np.nan)
ds["S_lambda"] = xr.where(mask, ds["S_lambda"], np.nan)

In [ ]:
ds.T.isel(time=0, Nz=-1).plot()

In [ ]:
ds.S.isel(time=0, Nz=-1).plot()

In [ ]:
ds = ds.fillna(-999)

In [ ]:
ds

In [ ]:
ds[["time", "T", "T_lambda", "S", "S_lambda"]].to_netcdf(os.path.join(dirpath, "Sogn_bry800.nc"))

In [ ]:
ds[["time", "T", "T_lambda", "S", "S_lambda"]]